In [3]:
import pandas as pd
import numpy as np

import scipy.stats

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
user_spec_fillna = pd.read_csv('user_spec_fillna.csv')
user_spec_fillna.head()

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,ymd
0,1031304,1,2000.0,1.0,2022-05-03 14:51:00,750.0,20000000.0,OTHERINCOME,NaN,기타,전월세,0.0,생활비,0.0,-1.0,1.0,10000000.0,2022-05-03
1,1767402,1,2000.0,1.0,2022-05-03 14:53:33,750.0,20000000.0,OTHERINCOME,NaN,기타,전월세,1000000.0,생활비,0.0,-1.0,1.0,10000000.0,2022-05-03
2,1096882,7,1985.0,1.0,2022-05-22 16:42:47,580.0,22000000.0,EARNEDINCOME2,201907.0,계약직,기타가족소유,10000000.0,생활비,0.0,-1.0,7.0,46000000.0,2022-05-22
3,248576,9,1996.0,1.0,2022-05-21 23:41:09,810.0,29000000.0,EARNEDINCOME,202201.0,계약직,기타가족소유,15000000.0,사업자금,0.0,-1.0,1.0,10000000.0,2022-05-21
4,1180616,11,1997.0,1.0,2022-03-24 10:58:22,640.0,25000000.0,EARNEDINCOME,202106.0,정규직,기타가족소유,2000000.0,생활비,0.0,-1.0,1.0,14000000.0,2022-03-24


#### ANOVA 분석을 위한 가정 검정
+ 정규성 : shapiro
+ 독립성
+ 등분산성 : levene, bartlett

01) 정규성 검정
+ 각 범주형 변수의 요소들의 데이터 수가 30이상이라 정규성 검정이 필요하지는 않음
+ 결과가 궁금해서 한 번 실행

In [16]:
cat_list = ['income_type','employment_type','houseown_type','purpose']#,'personal_rehabilitation_yn','personal_rehabilitation_complete_yn']

In [17]:
for cat in cat_list:
    print(user_spec_fillna.groupby(['%s'%cat]).count())

                 application_id  user_id  birth_year  gender  insert_time  \
income_type                                                                 
EARNEDINCOME             892629   892629      887669  887669       892629   
EARNEDINCOME2             78810    78810       78176   78176        78810   
FREELANCER               103914   103914      103083  103083       103914   
OTHERINCOME              172604   172604      170132  170132       172604   
PRACTITIONER              20126    20126       19995   19995        20126   
PRIVATEBUSINESS          126112   126112      125417  125417       126112   

                 credit_score  yearly_income  company_enter_month  \
income_type                                                         
EARNEDINCOME           834062         892629               892623   
EARNEDINCOME2           72574          78810                78810   
FREELANCER              94451         103914               103914   
OTHERINCOME            155386         

In [19]:
for cat in cat_list:
    print('shapiro : %s'%cat)
    for uni in user_spec_fillna[cat].unique():
        print('%s: '%uni, scipy.stats.shapiro(user_spec_fillna.credit_score[user_spec_fillna[cat]== uni]))
    print('')

shapiro : income_type
OTHERINCOME:  ShapiroResult(statistic=nan, pvalue=1.0)
EARNEDINCOME2:  ShapiroResult(statistic=nan, pvalue=1.0)
EARNEDINCOME:  ShapiroResult(statistic=nan, pvalue=1.0)
FREELANCER:  ShapiroResult(statistic=nan, pvalue=1.0)
PRIVATEBUSINESS:  ShapiroResult(statistic=nan, pvalue=1.0)
PRACTITIONER:  ShapiroResult(statistic=nan, pvalue=1.0)

shapiro : employment_type
기타:  ShapiroResult(statistic=nan, pvalue=1.0)
계약직:  ShapiroResult(statistic=nan, pvalue=1.0)
정규직:  ShapiroResult(statistic=nan, pvalue=1.0)
일용직:  ShapiroResult(statistic=nan, pvalue=1.0)

shapiro : houseown_type
전월세:  ShapiroResult(statistic=nan, pvalue=1.0)
기타가족소유:  ShapiroResult(statistic=nan, pvalue=1.0)
자가:  ShapiroResult(statistic=nan, pvalue=1.0)
배우자:  ShapiroResult(statistic=nan, pvalue=1.0)

shapiro : purpose
생활비:  ShapiroResult(statistic=nan, pvalue=1.0)
사업자금:  ShapiroResult(statistic=nan, pvalue=1.0)
대환대출:  ShapiroResult(statistic=nan, pvalue=1.0)
주택구입:  ShapiroResult(statistic=nan, pvalue=1.0)
SW

+ p-value > 0.05 이기 때문에 정규성 만족

#### 02) 독립성 검정
+ 자료 수집이 랜덤 샘플링이라면 만족한다고 봄

#### 03) 등분산성 검정
+ 모든 집단의 모분산은 동일하다 가정
+ 집단의 모분산의 유의미한 차이 파악
+ 귀무가설 : 해당 범주형 변수의 등분산은 모두 같다

bartlett
+ 귀무가설 : 분산의 차이가 있다
+ 대립가설 ; 분산의 차이가 없다

levene
+ 귀무가설 : 분산의 차이가 없다
+ 대립가설 : 분산의 차이가 있다

fligner
+ 귀무가설 : 분산의 차이가 없다
+ 대립가설 : 분산의 차이가 있다

In [27]:
cat = 'income_type'

scipy.stats.bartlett(
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[0]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[1]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[2]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[3]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[4]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[5]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[6]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[7]]
  )

BartlettResult(statistic=3128.0898694294265, pvalue=0.0)

In [30]:
cat = 'employment_type'

scipy.stats.fligner(
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[0]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[1]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[2]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[3]],
      #user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[4]],
      #user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[5]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[6]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[7]]
  )

FlignerResult(statistic=364983.87865481543, pvalue=0.0)

In [25]:
cat = 'houseown_type'

scipy.stats.levene(
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[0]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[1]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[2]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[3]],
      #user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[4]],
      #user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[5]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[6]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[7]]
  )

LeveneResult(statistic=nan, pvalue=nan)

In [ ]:
cat = 'income_type'

scipy.stats.levene(
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[0]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[1]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[2]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[3]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[4]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[5]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[6]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[7]]
  )

In [ ]:
cat = 'income_type'

scipy.stats.levene(
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[0]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[1]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[2]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[3]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[4]],
      user_spec_fillna.credit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[5]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[6]],
      #user_spec_fillnacredit_score[user_spec_fillna[cat] == user_spec_fillna[cat].unique()[7]]
  )